In [1]:
import wandb
import pandas as pd
import pandas_profiling

### Fetch the sample.csv artifact from W&B.

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: rpdippold (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Create a profile of the data using pandas-profiling

In [3]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Get a list of the counts and dtypes of all of the dataframe's columns

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

### Findings from the profile and info
1. **Overall**: The data file contains 20000 rows and 16 columns (variables). According to the profile there are 10 numeric and 6 categorical variables.
2. **Missing values**: There are missing values in columns `last_review`, `reviews_per_month`, `name` and `host_name`.
3. **Dtypes**: Column `last_review` contains dates but is regarded as type object.
4. **Outliers**: There seem to be outlies in columns `price`, `minimum_nights`, `number_of_reviews` and `calculated_host_listings_counts`.
5. **Zeros**: There are many zeros in columns `availability_365` and `number_of_reviews`.


### Fixing some problems in the data
- Drop outliers in column `price`
- Convert `last_review` to datetime
- Imputing missing values will be postponed to the inference pipeline.

In [13]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

In [14]:
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

### Check if problems have been solved

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu